### use llm for topic representation

In [1]:
import os,sys
sys.path.insert(0,'../libs')
from utils import load_json
import openai
from llm_utils import BSAgent_legacy
import numpy as np
from bertopic import BERTopic
import pandas as pd
import re,json,copy
from tqdm import tqdm
## load API Key
key = load_json('/home/chuang/Dev/Keys/openai_key.json') 
os.environ['OPENAI_API_KEY'] = key['ChatGPT']['API_KEY']
openai.api_key  = os.getenv('OPENAI_API_KEY')

...currently using old version of openai


- load legacy llm agent class, as fund server having problem with the new version

In [2]:
llm_agent  = BSAgent_legacy(model="gpt-3.5-turbo-1106", 
                    temperature=0)

In [3]:
## just run one test, make sure the api works 
pt = {'System':'You are a helpful assistant.',
      'Human':'What is your name?'}
res = llm_agent.get_response_content(prompt_template=pt)
print(res)  


I am a language model AI created by OpenAI, and I don't have a personal name. You can just call me "Assistant." How can I assist you today?


In [4]:
res_folder = '/data/chuang/news_scrape/data/news_search_res'
emb_path  = os.path.join(res_folder,'sentence_embeddings.npy')
docs_path = os.path.join(res_folder,'docs.npy')
topic_model_out_path = os.path.join(res_folder,'topic_model')

In [5]:
topic_model = BERTopic.load(topic_model_out_path)
topic_model.calculate_probabilities = False ## do not calculate probs for all topics 
## load training doc and embeddings 
embeddings = np.load(emb_path)
docs = np.load(docs_path)

In [6]:
topic_model.visualize_barchart()

In [7]:
delim = '\n----------------\n----------------\n'

topic_df = topic_model.get_topic_info()
topic_df['topic_words'] = topic_model.generate_topic_labels(nr_words=10,
                                                            topic_prefix=False,
                                                            word_length=50,
                                                            separator=", ")
rep_doc_dict = topic_model.get_representative_docs()      
rep_doc_dict = {k:delim.join([re.sub(r'\n+', '\n', text) for text in v]) for k,v in rep_doc_dict.items()} 
topic_df.head()

,Topic,Count,Name,topic_words
0,-1,727,-1_st vincent_vincentian_vincent grenadines_vi...,"st vincent, vincentian, vincent grenadines, vi..."
1,0,336,0_tournament_tennis_athletics_championship,"tournament, tennis, athletics, championship, c..."
2,1,304,1_accused_prosecution_magistrate_offences,"accused, prosecution, magistrate, offences, cr..."
3,2,237,2_teachers union_teachers_students_classroom,"teachers union, teachers, students, classroom,..."
4,3,113,3_party ndp_ndp_democratic party_new democratic,"party ndp, ndp, democratic party, new democrat..."


In [8]:
summary_pt = {'System':"You're a helpful assistant. Your task is to analyse hotel topics in news articles.",
      'Human':"""
Please summarize the following new article with less then 100 words: 
----------------
----------------

{DOCUMENTS}

----------------
----------------
Please write your summery below:
"""}

topic_pt = {'System':"You're a helpful assistant. Your task is to analyse hotel topics in news articles.",
      'Human':"""
I have a topic that contains the following documents delimited with ----------------: 
----------------
----------------

{DOCUMENTS}

----------------
----------------

The topic is described by the following keywords: {KEYWORDS}

Based on the information above, extract a short topic label, a list of topic words and a short descrption of the topic.
Make sure the label has no more than 3 words.
Make sure topic words arel less than 10 words.
And also makse sure the description is only a short highlevel sumamry.  
Please respond in clean json format as follow:
topic: <topic label>,
topic_words: [<topic workds>],
description: <topic description>

"""}
#topic_pt['Human'] = topic_pt['Human'].format(DOCUMENTS=rep_doc_dict[1],KEYWORDS=topic_df.iloc[1]['topic_words'])

In [13]:
llm_res = []
for k,v in tqdm(rep_doc_dict.items()):
    topic_pt_temp = copy.copy(topic_pt)
    topic_pt_temp['Human']=topic_pt_temp['Human'].format(DOCUMENTS=v,KEYWORDS=topic_df.iloc[k]['topic_words'])
    try:
        res = llm_agent.get_response_content(prompt_template=topic_pt_temp,conv_history=[])
        res_dict = llm_agent.parse_load_json_str(res)
    except:
        res_dict = {'topic':None,'topic_words':None,'description':None}
    #print(res_dict['topic'])
    res_dict['topic_id'] = k
    llm_res.append(res_dict)

  2%|▏         | 1/41 [00:01<01:08,  1.71s/it]

Community College English Program


  5%|▍         | 2/41 [00:03<00:59,  1.52s/it]

Vincentian Athletics


  7%|▋         | 3/41 [00:05<01:13,  1.93s/it]

...

St. Vincent Flooding


 98%|█████████▊| 40/41 [05:39<00:06,  6.42s/it]

Flooding in St Vincent


100%|██████████| 41/41 [05:40<00:00,  8.31s/it]

Water Supply Issues


In [26]:
llm_df = pd.DataFrame(llm_res)
llm_df=llm_df.sort_values('topic_id',ascending=True)
llm_df.columns = ['llm_topic','llm_topic_words','llm_description','llm_topic_id']
llm_df.head()

,llm_topic,llm_topic_words,llm_description,llm_topic_id
0,Community College English Program,"[Ecuadorean, teachers, English, Community Coll...",The initiative to teach Ecuadorean teachers En...,-1
1,Vincentian Athletics,"[St Vincent, Vincentians, athletics, Penn Rela...",Coverage of St Vincent and the Grenadines' par...,0
2,Mental Health Centre,"[psychiatric hospital, mental health, psychiat...",The court considers the report on model Yugge ...,1
3,School Reopening Concerns,"[School reopening, Teachers' Union, Profession...",The discussion and concerns surrounding the pr...,2
4,Opposition Leadership Transition,"[Opposition, Leadership, Transition, NDP, Conv...",The transition of leadership within the New De...,3


In [27]:
final_result = pd.merge(topic_df, llm_df, how='left',left_on='Topic',right_on='llm_topic_id')
final_result.head()

,Topic,Count,Name,topic_words,llm_topic,llm_topic_words,llm_description,llm_topic_id
0,-1,727,-1_st vincent_vincentian_vincent grenadines_vi...,"st vincent, vincentian, vincent grenadines, vi...",Community College English Program,"[Ecuadorean, teachers, English, Community Coll...",The initiative to teach Ecuadorean teachers En...,-1
1,0,336,0_tournament_tennis_athletics_championship,"tournament, tennis, athletics, championship, c...",Vincentian Athletics,"[St Vincent, Vincentians, athletics, Penn Rela...",Coverage of St Vincent and the Grenadines' par...,0
2,1,304,1_accused_prosecution_magistrate_offences,"accused, prosecution, magistrate, offences, cr...",Mental Health Centre,"[psychiatric hospital, mental health, psychiat...",The court considers the report on model Yugge ...,1
3,2,237,2_teachers union_teachers_students_classroom,"teachers union, teachers, students, classroom,...",School Reopening Concerns,"[School reopening, Teachers' Union, Profession...",The discussion and concerns surrounding the pr...,2
4,3,113,3_party ndp_ndp_democratic party_new democratic,"party ndp, ndp, democratic party, new democrat...",Opposition Leadership Transition,"[Opposition, Leadership, Transition, NDP, Conv...",The transition of leadership within the New De...,3


In [28]:
llm_fintune_out = os.path.join(res_folder,'llm_results.xlsx')
final_result.to_excel(llm_fintune_out)